In [123]:
!pip install QuantLib

In [124]:
import numpy as np
import pandas as pd
import QuantLib as ql
import datetime as dt


In [125]:
clean_prices = np.array([101+10/32+1/64, 100+18/32, 100+20/32+3/128, 100+23/32+3/128, 98+15/32+3/128, 88+19/32, 86+23/32, 80+23/32+1/64, 80+10/32+1/64, 81+18/32+1/64, 85+29/32+3/128, 83+5/32, 82+27/32+3/128, 94+1/32+1/64, 103+29/32+3/128, 83+7/32+1/128 ])

In [126]:
clean_prices

array([101.328125 , 100.5625   , 100.6484375, 100.7421875,  98.4921875,
        88.59375  ,  86.71875  ,  80.734375 ,  80.328125 ,  81.578125 ,
        85.9296875,  83.15625  ,  82.8671875,  94.046875 , 103.9296875,
        83.2265625])

In [127]:
bonds_tyh3 = pd.DataFrame()
bonds_tyh3["clean_prices"] = clean_prices
coupon_dates = [ql.Date(31,10,2022),ql.Date(30, 9, 2022),ql.Date(30,11,2022),ql.Date(31,12,2022),ql.Date(31,1,2023),ql.Date(15,11,2022),ql.Date(15,2,2022),ql.Date(15,5,2022),ql.Date(15,8,2022),ql.Date(15,11,2022),ql.Date(15,5,2022),ql.Date(15,2,2022),ql.Date(15,8,2022),ql.Date(15,5,2022),ql.Date(15,11,2022),ql.Date(15,11,2022)]
coupon_dates = np.array(coupon_dates)
eval_date = ql.Date(7,2,2023)
period = ql.Period("6M")
first_coupon_dates  = coupon_dates + period
days_between = first_coupon_dates - coupon_dates
bonds_tyh3["coupon_dates"] = coupon_dates
bonds_tyh3["first_coupon_dates"] = first_coupon_dates
bonds_tyh3["eval_date"] = eval_date
bonds_tyh3["days_between"] = days_between
coupon_rate = np.array([4, 3+7/8, 3+7/8, 3+7/8, 3.5, 1.75, 1.5, 5/8, 5/8, 7/8, 1+5/8, 1+1/8, 1+1/4, 2+7/8, 4+1/8, 1+3/8])
bonds_tyh3["coupon_rate"] = coupon_rate
accured = lambda eval_date, coupon_date, days_between, r: (eval_date - coupon_date)/days_between*r/2
bonds_tyh3["accured"] = accured(bonds_tyh3.eval_date, bonds_tyh3.coupon_dates,bonds_tyh3.days_between, bonds_tyh3.coupon_rate)
bonds_tyh3["dirty_prices"]  = bonds_tyh3.clean_prices + bonds_tyh3.accured
bonds_tyh3["first_day_dlv"] = ql.Date(1,3,2023)
bonds_tyh3["delivery"] = ql.Date(31,3,2023)
bonds_tyh3["days_til_dlv"] = bonds_tyh3["delivery"] - bonds_tyh3["eval_date"]
bonds_tyh3["semi_ann_coupon"] = 100000* bonds_tyh3["coupon_rate"]/2
bonds_tyh3["ai_until_delivery"] = bonds_tyh3["days_til_dlv"]/ bonds_tyh3["days_between"] * bonds_tyh3["semi_ann_coupon"]

In [128]:

bonds_tyh3["carry"] =  (bonds_tyh3['days_til_dlv'] / bonds_tyh3['days_between']) * bonds_tyh3['coupon_rate']/2
# carry = (delivery / period) * coupon  / 2

actual_repo = 0.04597
bonds_tyh3["repo_cost"] = actual_repo /360 * bonds_tyh3["days_til_dlv"] *bonds_tyh3['dirty_prices']

In [129]:
exp_dates = [ql.Date(31,10,2029), ql.Date(30,9,2029),ql.Date(30,11,2029),ql.Date(31,12,2029),ql.Date(31,1,2030),ql.Date(15,11,2029),ql.Date(15,2,2030),ql.Date(15,5,2030),\
    ql.Date(15,8,2030), ql.Date(15,11,2030), ql.Date(15,5,2031), ql.Date(15,2,2031), ql.Date(15,8,2031), ql.Date(15,5,2032), ql.Date(15,11,2032), ql.Date(15,11,2031)]

bonds_tyh3["exp"] = exp_dates

In [130]:
from math import trunc
from math import floor
n = []
for i in range(len(bonds_tyh3["exp"])):
    n += [floor((bonds_tyh3["exp"].iloc[i] - bonds_tyh3["first_day_dlv"].iloc[i])/365)]
bonds_tyh3["n"] = n
n

[6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 7, 8, 9, 9, 8]

In [131]:
z_po = []
for i in range (len(bonds_tyh3["exp"])):
    if (bonds_tyh3["exp"].iloc[i].dayOfMonth() > bonds_tyh3["first_day_dlv"].iloc[i].dayOfMonth()):
        if (bonds_tyh3 ["exp"].iloc[i].month() > bonds_tyh3["first_day_dlv"].iloc[i].month()):
            z_po += [floor((bonds_tyh3["exp"].iloc[i].month() - bonds_tyh3["first_day_dlv"].iloc[i].month())/3)]
        else:
            z_po += [floor((12 + bonds_tyh3["exp"].iloc[i].month() - bonds_tyh3["first_day_dlv"].iloc[i].month())/3)]
    else:
        if (bonds_tyh3 ["exp"].iloc[i].month() > bonds_tyh3["first_day_dlv"].iloc[i].month()):
            z_po += [floor((bonds_tyh3["exp"].iloc[i].month() - bonds_tyh3["first_day_dlv"].iloc[i].month()-1)/3) ]
        else:
            z_po += [floor((12 + bonds_tyh3["exp"].iloc[i].month() - bonds_tyh3["first_day_dlv"].iloc[i].month())/3)]
z = np.array(z_po)*3

bonds_tyh3["z"] = z

In [132]:
v = []
for i in range (len(bonds_tyh3["exp"])):
    if bonds_tyh3["z"].iloc[i] < 7:
        v += [bonds_tyh3["z"].iloc[i]]
    elif bonds_tyh3["z"].iloc[i] >= 7:
        v += [bonds_tyh3["z"].iloc[i] - 6]
bonds_tyh3["v"] = v

In [133]:
def cf(coup,n,z,v):
    a = 1/1.03**(v/6)
    b = (coup/2)*(6-v)/6
    if z < 7:
        c = 1/1.03**(2*n)
    else:
        c = 1/1.03**(2*n+1)
    d = (coup/0.06)*(1-c)

    return a*((coup/2) +c +d )-b



In [134]:
cfs = []
for i in range (len(bonds_tyh3["exp"])):
    cfs += [round(cf(bonds_tyh3["coupon_rate"].iloc[i]/100, bonds_tyh3["n"].iloc[i], bonds_tyh3["z"].iloc[i], bonds_tyh3["v"].iloc[i]),4)]
bonds_tyh3["Convert Factor"] = cfs

In [135]:
gb = lambda clean_price, cf : clean_price - (113 + 16/32) * cf

# net_basis = gross_basis - (carry - repo_cost)
nb = lambda gross_basis, carry, repo_cost: gross_basis - (carry - repo_cost)
bonds_tyh3["gross_basis"] = gb(bonds_tyh3["clean_prices"], bonds_tyh3["Convert Factor"])
bonds_tyh3["G32"] = bonds_tyh3["gross_basis"]*32

bonds_tyh3["net_basis"] = bonds_tyh3["gross_basis"] - (bonds_tyh3['carry'] - bonds_tyh3["repo_cost"])
bonds_tyh3["N32"] = bonds_tyh3["net_basis"]* 32

In [136]:
bonds_tyh3['gross_basis']

0    -0.106825
1    -0.112000
2    -0.026062
3     0.476287
4     0.564387
5     0.744750
6     1.230550
7     1.692975
8     2.274175
9     2.797775
10    3.619488
11    3.558700
12    4.052787
13    4.938025
14    5.672737
15    4.287312
Name: gross_basis, dtype: float64

In [137]:
bonds_tyh3["implied_repo"] = -(bonds_tyh3["gross_basis"]*100000 - bonds_tyh3["ai_until_delivery"])/(bonds_tyh3["days_til_dlv"]/360* bonds_tyh3["dirty_prices"]*100000)

In [138]:
bonds_tyh3

,clean_prices,coupon_dates,first_coupon_dates,eval_date,days_between,coupon_rate,accured,dirty_prices,first_day_dlv,delivery,...,exp,n,z,v,Convert Factor,gross_basis,G32,net_basis,N32,implied_repo
0,101.328125,"October 31st, 2022","April 30th, 2023","February 7th, 2023",181,4.000,1.093923,102.422048,"March 1st, 2023","March 31st, 2023",...,"October 31st, 2029",6,6,6,0.8937,-0.106825,-3.4184,-0.001317,-0.042139,0.046059
1,100.562500,"September 30th, 2022","March 30th, 2023","February 7th, 2023",181,3.875,1.391575,101.954075,"March 1st, 2023","March 31st, 2023",...,"September 30th, 2029",6,6,6,0.8870,-0.112000,-3.5840,0.008357,0.26741,0.045403
2,100.648438,"November 30th, 2022","May 30th, 2023","February 7th, 2023",181,3.875,0.738605,101.387042,"March 1st, 2023","March 31st, 2023",...,"November 30th, 2029",6,6,6,0.8870,-0.026062,-0.8340,0.090529,2.896925,0.039788
3,100.742188,"December 31st, 2022","June 30th, 2023","February 7th, 2023",181,3.875,0.406768,101.148955,"March 1st, 2023","March 31st, 2023",...,"December 31st, 2029",6,9,3,0.8834,0.476287,15.2412,0.591298,18.921535,0.005499
4,98.492188,"January 31st, 2023","July 31st, 2023","February 7th, 2023",181,3.500,0.06768,98.559867,"March 1st, 2023","March 31st, 2023",...,"January 31st, 2030",6,9,3,0.8628,0.564387,18.0604,0.716074,22.914353,-0.004329
5,88.593750,"November 15th, 2022","May 15th, 2023","February 7th, 2023",181,1.750,0.406077,88.999827,"March 1st, 2023","March 31st, 2023",...,"November 15th, 2029",6,6,6,0.7740,0.744750,23.8320,1.084338,34.698801,-0.038378
6,86.718750,"February 15th, 2022","August 15th, 2022","February 7th, 2023",181,1.500,1.479282,88.198032,"March 1st, 2023","March 31st, 2023",...,"February 15th, 2030",6,9,3,0.7532,1.230550,39.3776,1.600725,51.223204,-0.079678
7,80.734375,"May 15th, 2022","November 15th, 2022","February 7th, 2023",184,0.625,0.455163,81.189538,"March 1st, 2023","March 31st, 2023",...,"May 15th, 2030",7,0,0,0.6964,1.692975,54.1752,2.143767,68.600555,-0.13683
8,80.328125,"August 15th, 2022","February 15th, 2023","February 7th, 2023",184,0.625,0.298913,80.627038,"March 1st, 2023","March 31st, 2023",...,"August 15th, 2030",7,3,3,0.6877,2.274175,72.7736,2.721232,87.079433,-0.18769
9,81.578125,"November 15th, 2022","May 15th, 2023","February 7th, 2023",181,0.875,0.203039,81.781164,"March 1st, 2023","March 31st, 2023",...,"November 15th, 2030",7,6,6,0.6941,2.797775,89.5288,3.21512,102.883853,-0.226202


In [139]:
bonds_tyh3["implied_repo"] = bonds_tyh3["implied_repo"] *100
output_table = bonds_tyh3[["Convert Factor","G32","N32","implied_repo"]]
output_table

,Convert Factor,G32,N32,implied_repo
0,0.8937,-3.4184,-0.042139,4.605901
1,0.8870,-3.5840,0.26741,4.540256
2,0.8870,-0.8340,2.896925,3.978836
3,0.8834,15.2412,18.921535,0.549898
4,0.8628,18.0604,22.914353,-0.432869
5,0.7740,23.8320,34.698801,-3.837794
6,0.7532,39.3776,51.223204,-7.967842
7,0.6964,54.1752,68.600555,-13.683023
8,0.6877,72.7736,87.079433,-18.768983
9,0.6941,89.5288,102.883853,-22.620179


In [140]:
table = pd.read_excel("Bond Future DLV Data.xlsx")
table


,Cash Security,IssueDate,Price,Conver Factor,Gro/Bas (32nds)
0,T 4 10/31/29,2022-10-31,101-10+,0.8937,-3.4176
1,T 3 ⅞ 09/30/29,2022-09-30,100-18,0.8870,-3.5840
2,T 3 ⅞ 11/30/29,2022-11-30,100-20 3/4,0.8870,-0.8336
3,T 3 ⅞ 12/31/29,2023-01-03,100-23 3/4,0.8834,NaN
4,T 3 ½ 01/31/30,2023-01-31,98-15 3/4,0.8628,NaN
5,T 1 ¾ 11/15/29,2019-11-15,88-19,0.7740,NaN
6,T 1 ½ 02/15/30,2020-02-18,86-23,0.7532,NaN
7,T 0 ⅝ 05/15/30,2020-05-15,80-23+,0.6964,NaN
8,T 0 ⅝ 08/15/30,2020-08-17,80-10+,0.6877,NaN
9,T 0 ⅞ 11/15/30,2020-11-16,81-18+,0.6941,NaN


In [143]:
output_table[['Cash Security',"IssueDate","Price"]] = table[['Cash Security',"IssueDate","Price"]]
output_table2 = output_table[['Cash Security',"IssueDate","Price","Convert Factor","G32","N32","implied_repo"]]
output_table2

<ipython-input-143-01f7833f8df5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_table[['Cash Security',"IssueDate","Price"]] = table[['Cash Security',"IssueDate","Price"]]


,Cash Security,IssueDate,Price,Convert Factor,G32,N32,implied_repo
0,T 4 10/31/29,2022-10-31,101-10+,0.8937,-3.4184,-0.042139,4.605901
1,T 3 ⅞ 09/30/29,2022-09-30,100-18,0.8870,-3.5840,0.26741,4.540256
2,T 3 ⅞ 11/30/29,2022-11-30,100-20 3/4,0.8870,-0.8340,2.896925,3.978836
3,T 3 ⅞ 12/31/29,2023-01-03,100-23 3/4,0.8834,15.2412,18.921535,0.549898
4,T 3 ½ 01/31/30,2023-01-31,98-15 3/4,0.8628,18.0604,22.914353,-0.432869
5,T 1 ¾ 11/15/29,2019-11-15,88-19,0.7740,23.8320,34.698801,-3.837794
6,T 1 ½ 02/15/30,2020-02-18,86-23,0.7532,39.3776,51.223204,-7.967842
7,T 0 ⅝ 05/15/30,2020-05-15,80-23+,0.6964,54.1752,68.600555,-13.683023
8,T 0 ⅝ 08/15/30,2020-08-17,80-10+,0.6877,72.7736,87.079433,-18.768983
9,T 0 ⅞ 11/15/30,2020-11-16,81-18+,0.6941,89.5288,102.883853,-22.620179


In [144]:
output_table2.to_excel('Project 4 Output Table.xlsx')